In [17]:
# %load trainer.py
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchsummary
import torch.optim as optim
from torch.utils.data import DataLoader


In [21]:

train_transform_with_RE = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224), # 从中心裁剪
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing()
])
train_transform_noRE = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224), # 从中心裁剪
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
#load cat and dog image as trainig data
train_dataset_with_RE = datasets.ImageFolder(root="Q2_Dataset/Dataset_OpenCvDl_Hw2_Q5/training_dataset", transform=train_transform_with_RE)
train_dataset_noRE = datasets.ImageFolder(root="Q2_Dataset/Dataset_OpenCvDl_Hw2_Q5/training_dataset", transform=train_transform_noRE)
#print num of traing dataset
# print(train_dataset.__len__)
test_dataset = datasets.ImageFolder(root="Q2_Dataset/Dataset_OpenCvDl_Hw2_Q5/validation_dataset", transform=test_transform)
# print(test_dataset.class_to_idx)
# print(test_dataset.__len__)

#load training data
train_loader_with_RE = DataLoader(dataset=train_dataset_with_RE, batch_size=64, shuffle=True)
train_loader_noRE = DataLoader(dataset=train_dataset_noRE, batch_size=64, shuffle=True)

#load testing data
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

In [ ]:
#model
resnet = models.resnet50(pretrained=True)
#Replace the output layer to a FC (Fully Connected) layer of 2 node with a Softmax activation function
resnet.fc = nn.Sequential(nn.Linear(resnet.fc.in_features, 2), nn.Softmax(dim=1))
torchsummary.summary(resnet, (3, 224, 224))

/Users/yiting/Course/cv2024/env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yiting/Course/cv2024/env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/yiting/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [02:31<00:00, 677kB/s] 


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

/Users/yiting/Course/cv2024/env/lib/python3.12/site-packages/torch/nn/modules/module.py:1844: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return inner()


In [ ]:
#device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)
print(device)

cpu


In [19]:
##tranmodel 
def train_model(model, train_loader, test_loader, num_epochs=30, learning_rate=0.001,save_model_name = 'best_model.ckpt'):

    print(f"using device: {device}")


    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)


    # Initialize lists to save metrics
    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []

    # Variable to track the best accuracy
    best_accuracy = 0.0

    # Train the model
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        train_loss = 0.0
        total_train = 0
        correct_train = 0

        for images, labels in train_loader:
            # No need for Variable, tensors work directly
            images, labels = images.to(device), labels.to(device)  # Move to GPU if available

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item() * images.size(0)  # Accumulate batch loss

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate training accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        # Calculate average training loss and accuracy
        train_loss /= len(train_loader.dataset)
        train_accuracy = 100 * correct_train / total_train

        # Evaluate the model on the validation set
        model.eval()  # Set model to evaluation mode
        valid_loss = 0.0
        correct_test = 0
        total_test = 0

        with torch.no_grad():  # Disable gradient computation for testing
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)  # Move to GPU if available

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                valid_loss += loss.item() * images.size(0)  # Accumulate batch loss

                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        valid_loss /= len(test_loader.dataset)
        valid_accuracy = 100 * correct_test / total_test

        # Save metrics for the current epoch
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        train_accuracies.append(train_accuracy)
        valid_accuracies.append(valid_accuracy)

        # Output training and testing results
        print(f"Epoch [{epoch+1}/{num_epochs}], ")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
        print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%\n")

        # Save the best model based on validation accuracy
        if valid_accuracy > best_accuracy:
            best_accuracy = valid_accuracy
            torch.save(model.state_dict(), save_model_name)
            print(f"Best model saved with accuracy: {best_accuracy:.2f}%")

    # # Print final results
    # print("Training Losses:", train_losses)
    # print("Validation Losses:", valid_losses)
    # print("Training Accuracies:", train_accuracies)
    # print("Validation Accuracies:", valid_accuracies)

In [22]:
#train model with RE
train_model(resnet, train_loader_with_RE, test_loader, num_epochs=30, learning_rate=0.001,save_model_name = 'best_model_with_RE.ckpt')

using device: cpu


/Users/yiting/Course/cv2024/env/lib/python3.12/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


KeyboardInterrupt: 